# Prelude

In [ ]:
# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:\n")
!git status --short

# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from bgcellmodels.common import analysis
from bgcellmodels.mechanisms import noise, synapses

import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# Create cell

In [ ]:
# Load cell model
from bgcellmodels.models.interneuron.Golomb2007_FSI import golomb_pynn_model
icell = h.GolombFSI()
soma = icell.soma[0]

# Fix ion channel density to Golomb (2007) value
icell.set_gbar_delayed_tonic()

# Set error tolerances for adaptive integrator
h.golomb_set_state_tolerances()

# CTX Afferents

## Required Data

Calibration of Cortex - FSI afferents.

Required data:
- number of synapses
- presynaptic firing rate + pattern
- magnitude of EPSC / EPSP
- strength and timescale of STD

## Experimental Data


### Number & Location

Bennett, Bolam (1994): 

> The synaptic input and output of parvalbumin-  immunoreactive neurons was examined to determine  if the connectivity of this population of cells could  account for feed-forward inhibition, detected electrophysiologically, in spiny projection neurons following cortical activation [42,59]. The cortex was found  to directly innervate the perikarya and dendrites of  parvalbumin-immunopositive neurons and that this  class of neuron, in turn, was found to make synaptic  contact, via terminals that are GABA-positive, with  striatonigral projection neurons. Together. these data  demonstrate a synaptic circuit that may account for  inhibitory responses that occur in striatal neurons  following stimulation of the cortex.

> The spatial organization of cortical inputs to  parvalbumin-immunopositive neurons differs from  the arrangement of cortically-derived afferents of  spiny neurons and possibly cholinergic neurons. Spiny neurons receive cortical input to dendritic spines [15,26,51] and cholinergic cells are probably innervated in the distal dendritic regions [34,58].  In contrast, __the cell bodies as well as proximal and distal dendrites  of parvalbumin-containing neurons receive cortical  input__. These findings provide a possible explanation  for the observation that __cortical stimulation that is  subthreshold with respect to producing an EPSP in  spiny cells is sufficient to produce an IPSP__, pre-  sumably mediated by the firing of GABAergic internerons [59]. Hence, __it seems that GABAergic  interneurons are more sensitive to cortical input than  spiny neurons__, which may reflect the spatial distribution of the excitatory inputs on the cell bodies and  dendrites. Other factors are presumably also of im-  portance, such as their intrinsic electrical properties.  The greater sensitivity of GABAergic interneurons  may serve as a gating mechanism for the cortical  activation of spiny cells. Furthermore, when cortical  excitation is sufficient to elicit an EPSP in spiny cells, an IPSP resulting from the stimulation of GABAergic interneurons may confer temporal specificity to the  firing of spiny neurons by returning them to a state  of electrical quiescence.


### Synaptic Response

- Gittis (2010) Fig.3, Section 3
    + Fig 3.A shows EPSC
    + Fig 3.F shows `tau_decay ~= 1.4 ms`
    + Fig 3.B shows paired-pulse ratio
        - facilitation if two spikes within 50 ms (`PPR ~= 1.25`)
        - depression if two spikes wihin > 50 ms (`PPR ~= 0.75`)

### Firing Pattern

- 1-5 Hz during relaxed neutral posture (Bergman book chapter)
    + Poisson-like discharge rate with a slight tendency toward bursting. 

## Calibration Results

- Following parameters yield facilitation in first 50 ms, with slight depression after:
    + syn.tau_rec = 100.0
    + syn.tau_facil = 250.0
    + syn.U1 = P_release = 0.1

- For PSP magnitude:
    + `syn.gmax_AMPA = 0.6e-4` yields EPSP of ~ 1 mV

## CTX -> FSI (AMPA)

In [ ]:
# Calibrate AMPA synapse to get desired PSP time course

# Save inputs
stim_data = {}
istim = None

def make_ctx_afferents(num_pre=1, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       scale_gsyn=1.0, intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0):
    # Clear previous inputs
    stim_data['CTX'] = {}
    stim_data['CTX']['synapses'] = []
    stim_data['CTX']['netstims'] = []
    stim_data['CTX']['netcons'] = []
    
    # Make new inputs
    for i in range(num_pre):
        seg = soma(0.5)
        syn = h.GLUsyn(seg)
        stim_data['CTX']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate**-1*1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        stim_data['CTX']['netstims'].append(stim)
        stim_data['CTX']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart
            bstim.noise = burst_noise
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            stim_data['CTX']['netstims'].append(bstim)
            stim_data['CTX']['netcons'].append(bnc)
        
        # Common STP parameters
        syn.tau_rec = 100.0
        syn.tau_facil = 250.0
        syn.U1 = P_release = 0.1 # release probability
        
        # AMPA synapse parameters
        syn.gmax_AMPA = ctx_gAMPA = 0.6e-4 * scale_gsyn # [uS] or set netcon weight in [nS]
        syn.tau_r_AMPA = 0.5
        syn.tau_d_AMPA = 1.4

        # NMDA synapse parameters
        syn.gmax_NMDA = 0.0 # 0.25 * ctx_gAMPA # conductance_nS * peak_factor * 1e-3
        syn.tau_r_NMDA = 3.7
        syn.tau_d_NMDA = 80

make_ctx_afferents(num_pre=1, rate=80, noise=0, num_spikes=100)

In [ ]:
# Prevent spiking: hyperpolarizing current (like in article figure)
if istim is None:
    istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = 0.0 # 100 pA = 0.1 nA

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'synGLU': stim_data['CTX']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    # Synapse variables
    'gAMPA': {'pointp':'synGLU', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synGLU', 'var':'g_NMDA'},
    'iAMPA': {'pointp':'synGLU', 'var':'i_AMPA'},
    'iNMDA': {'pointp':'synGLU', 'var':'i_NMDA'},
    'iGLU': {'pointp':'synGLU', 'var':'i'},
    'Rrp': {'pointp':'synGLU', 'var':'R'},
    'Use': {'pointp':'synGLU', 'var':'Use'},
}

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -70.038
h.tstop = 2000.0
h.init()
h.run()
# nrnsim.run(h.tstop, h.dt)

In [ ]:
# Plot recorded traces
plot_interval = (700, 1000)

plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
plt.ylim((-72, -60))
plt.xlim(plot_interval)
plt.grid(True)

In [ ]:
# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gAMPA'].as_numpy(), 'b', label='gAMPA')
ax.plot(t_soma, vec_dict['gNMDA'].as_numpy(), 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.set_xlim(plot_interval)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iAMPA'].as_numpy(), 'b', label='iAMPA')
ax.plot(t_soma, vec_dict['iNMDA'].as_numpy(), 'r', label='iNMDA')
ax.plot(t_soma, vec_dict['iGLU'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.set_xlim(plot_interval)
ax.legend()

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True, timeRange=plot_interval) # yRange=(-80,40), timeRange=

# GPE Afferents



## Experimental Data

### Connection Patter, Number, Location

- Corbit, Whalen (2016), Table 3 + Fig. 1
    + 100% of FSI receive inputs from GPe neurons
    
    
- Bevan (1998), Table 3
    + a single GPe axon makes on average 6.7 contacts with one FSI neuron
    + 19-66% of GPe (Arkypallidal) contacts are onto PV+ FSI
    + 2.4-11.6% of GPe (Arkypallidal) contacts are onto the _soma_ of PV+ FSI
    + 14.9-57.6% of GPe (Arkypallidal) contacts are onto the _dendrite_ of PV+ FSI


### Synaptic Response

- Corbit, Whalen (2016), Table 3 + Fig. 1
    + in _CONTROL_: `Ipeak = 565.9+/-560.2 pA`
    + in _CONTROL_: `tau_d = 4.5+/-1.8 ms`
    + in _DD_: `Ipeak = 843.8+/-643.8 pA`
    + in _DD_: `tau_d = 5.0+/-6.0 ms`

### Firing Pattern

- Mallet (2016)
    + GPe Proto cells fired regularly at high rates (47.3+/-6.1 Hz) while GPe Arky cells (projecting to Str) were more irregularly active with lower awake firing rates (8.9+/-1.9 Hz)
        - => `rate = 8.9 +/- 1.9 Hz`


## Calibration Results

- tau_d: used experimental values
- STP: no data, used same as FSI-FSI
- gsyn: used same ratio of `gsyn_GPE_FSI / gsyn_FSI_FSI = 2.4` as in Corbit, Whalen (2016)

## GPE -> FSI (GABAA)

In [ ]:
# Calibrate GABAA conductance so we get PSP of desired magnitude
stim_data = {}
istim = None

def make_gpe_afferents(num_pre=1, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       scale_gsyn=1.0, intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0):
    # Clear previous inputs
    stim_data['GPE'] = {}
    stim_data['GPE']['synapses'] = []
    stim_data['GPE']['netstims'] = []
    stim_data['GPE']['netcons'] = []
    
    for i in range(num_pre):
        seg = soma(0.5)
        syn = h.GABAsyn(seg)
        stim_data['GPE']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate ** -1 * 1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        stim_data['GPE']['netstims'].append(stim)
        stim_data['GPE']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart
            bstim.noise = burst_noise
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            stim_data['GPE']['netstims'].append(bstim)
            stim_data['GPE']['netcons'].append(bnc)
        
        # Common STP parameters
        syn.tau_rec = 200.0
        syn.tau_facil = 0.5
        syn.U1 = P_release = 0.35 # release probability

        # GABAA synapse parameters
        syn.gmax_GABAA = 4.8e-5 * scale_gsyn # [uS] or set netcon weight in [nS]
        syn.tau_r_GABAA = 0.75
        syn.tau_d_GABAA = 4.5

        # GABAB synapse parameters
        syn.gmax_GABAB = 0.0 # conductance_nS * peak_factor * 1e-3
        syn.tau_r_GABAB = 5.0
        syn.tau_d_GABAB = 80

make_gpe_afferents(1, rate=50, noise=0, num_spikes=10)

In [ ]:
# Bring cell in state where we can see effect of PSPs

# make sure cell is depolarized
if istim is None:
    istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = 0.005 # 100 pA = 0.1 nA

# remove NaF channels
for seg in soma:
    seg.gnabar_Nam = 0.0

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'synGABA': stim_data['MSN']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -70.038
h.tstop = 2000.0
h.init()
h.run()

In [ ]:
# Plot recorded traces
plot_interval = (700, 1000)

plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
# plt.ylim((-90, -70))
plt.xlim(plot_interval)
plt.grid(True)

In [ ]:
# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gAMPA'].as_numpy(), 'b', label='gAMPA')
ax.plot(t_soma, vec_dict['gNMDA'].as_numpy(), 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.set_xlim(plot_interval)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iAMPA'].as_numpy(), 'b', label='iAMPA')
ax.plot(t_soma, vec_dict['iNMDA'].as_numpy(), 'r', label='iNMDA')
ax.plot(t_soma, vec_dict['iGLU'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.set_xlim(plot_interval)
ax.legend()

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True, timeRange=plot_interval) # yRange=(-80,40), timeRange=

# FSI Afferents

Calibration of afferents from other FSI.

The following data is required to calibrate synapse parameters:
- number of synapses
- presynaptic firing rate + pattern
- magnitude of IPSC / IPSP
- strength and timescale of STD

## Experimental Data

Sources:

- _Gittis (2010)_ is source used by Corbit, Whalen et al. (2016) for FSI-FSI connections

> In addition to GABAergic projections onto MSNs, FS interneurons also formed GABAergic synapses onto other FS interneurons, but not PLTS or cholinergic interneurons (Fig. 5D,E). In these experiments, interneurons were recorded with a KMeSO3 internal solution containing a high [Cl-]i to set ECl = -20 mV. This enabled identification of FS interneurons based on voltage-clamp and current-clamp param- eters while increasing the driving force of Cl- to better detect GABAergic synaptic transmission. The probability of finding a synaptic connection between pairs of FS interneurons was 58% (7/12), and 3 of 4 connected pairs were reciprocally connected. The conduc- tance of FS-FS synapses was significantly lower than that of FS-MSN synapses (1.1 +/- 1.5 nS, n = 6 vs 7.8 +/- 11.2 nS, n = 69; p = 0.009) (Fig. 5F) but the two types of synapses had similar decay kinetics (5.7 +/- 2.7 ms vs 8.0 +/- 3.2 ms; p = 0.1) and paired-pulse ratios (0.69 +/- 0.29 vs 0.68 +/- 0.22; p = 0.83) (Fig. 5G,H).


### Connection: Number, Location, Pattern

- Gittis (2010) Fig. 5, Section 5 text
    + Pconn = 0.58
    + Pconn_reciprocal = 0.75 * Pconn

### Synaptic Response

- Gittis (2010) Fig. 5, Section 5
    + gsyn_fsi_fsi = 0.1 * gsyn_fsi_msn (Fig. 5.F)
        + if PSP for FSI->MSN is between 0.75 - 4 mV (Tepper, Wilson 2007) and input resistance is approx. the same in MSN and FSI, the PSP for FSI->FSI should be between 0.075 - 0.4
    + tau_d = 5.7 +/- 2.7 ms 
    + PPR = 0.69 +/- 0.29 (STD similar to FSI-MSN synapse)
    

### Firing Pattern

- Gittis (2010)
    + table 1: max firing rate is 286 +/- 88 Hz
    + Methods: Inhibitory frequencies were based on the assumption that:
        - any given FS interneuron operates across a __dynamic range of 10 - 100 Hz__ (Berke et al., 2004; Berke, 2008)
    + Results: Recordings from the striatum of awake behaving mice reveal that FS interneurons express __average firing rates of 10 - 15 Hz__ and __modulate their firing rates up to 60 - 80 Hz__ during behavioral tasks (Berke et al., 2004; Berke, 2008)


- Tepper, Wilson (2007)
    + The neurons exhibit a very hyperpolarized membrane potential in vitro and do not fire spontaneously. When stimulated with a series of increasing amplitude depolarizing pulses, the neurons do not fire at all below a certain threshold. Over this threshold a tiny increment in stimulus amplitude results in a short burst of a few spikes. Further increases lead to episodes of high frequency firing interspersed with silent periods. Sufficiently strong stimuli evoke sustained firing at rates > 200 Hz (Kawaguchi, 1993; Kawaguchi et al., 1995; Koós and Tepper, 1999, 002; Bracci et al., 2002; Plotkin et al., 2005; Taverna et al., 2007)

## Calibration Results

- gbar was set to have PSP of 10% magnitude of that for FSI->MSN

## FSI -> FSI (GABAA)

In [ ]:
# Calibrate GABAA conductance so we get PSP of desired magnitude

stim_data = {}
istim = None

def make_fsi_afferents(num_pre=1, rate=50, noise=0, num_spikes=1e9, tstart=750, 
                       scale_gsyn=1.0, intraburst_rate=0, interburst_interval=200, 
                       burst_duration=50.0, burst_noise=1.0):
    # Clear previous inputs
    stim_data['FSI'] = {}
    stim_data['FSI']['synapses'] = []
    stim_data['FSI']['netstims'] = []
    stim_data['FSI']['netcons'] = []
    
    # Make new inputs
    for i in range(num_pre):
        seg = soma(0.5)
        syn = h.GABAsyn(seg)
        stim_data['FSI']['synapses'].append(syn)
        
        # One spike generator for background spikes
        stim = h.NetStim()
        stim.interval = rate ** -1 * 1e3
        stim.number = num_spikes
        stim.noise = noise
        stim.start = tstart
        nc = h.NetCon(stim, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        stim_data['FSI']['netstims'].append(stim)
        stim_data['FSI']['netcons'].append(nc)
        
        # One spike generator for bursts (does not spike between bursts)
        if intraburst_rate != 0:
            bstim = h.BurstStim()
            bstim.fast_invl = intraburst_rate ** -1 * 1e3
            bstim.slow_invl = interburst_interval
            bstim.burst_len = burst_duration * 1e-3 * intraburst_rate
            bstim.start = tstart
            bstim.noise = burst_noise
            bnc = h.NetCon(bstim, syn)
            bnc.delay = 1.0
            bnc.weight[0] = 1
            stim_data['FSI']['netstims'].append(bstim)
            stim_data['FSI']['netcons'].append(bnc)

        # Common STP parameters
        syn.tau_rec = 200.0
        syn.tau_facil = 0.5
        syn.U1 = P_release = 0.35 # release probability

        # AMPA synapse parameters
        syn.gmax_GABAA = 2e-5 * scale_gsyn # [uS] or set netcon weight in [nS]
        syn.tau_r_GABAA = 0.5
        syn.tau_d_GABAA = 5.7

        # NMDA synapse parameters
        syn.gmax_GABAB = 0.0 # conductance_nS * peak_factor * 1e-3
        syn.tau_r_GABAB = 5.0
        syn.tau_d_GABAB = 80


make_fsi_afferents(1, rate=50, noise=0, num_spikes=10)

In [ ]:
# Bring cell in state where we can see effect of PSPs

# make sure cell is depolarized
if istim is None:
    istim = h.IClamp(soma(0.5))
istim.delay = 250
istim.dur = 1000
istim.amp = 0.005 # 100 pA = 0.1 nA

# remove NaF channels
for seg in soma:
    seg.gna_NasFSI = 0.0

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'synGABA': stim_data['FSI']['synapses'][0],
}

trace_specs = {
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    # Synapse variables
    'gGABAA': {'pointp':'synGABA', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synGABA', 'var':'g_GABAB'},
    'Rrp': {'pointp':'synGABA', 'var':'Rrp'},
    'Use': {'pointp':'synGABA', 'var':'Use'},
    'iGABAA': {'pointp':'synGABA', 'var':'i_GABAA'},
    'iGABAB': {'pointp':'synGABA', 'var':'i_GABAB'},
    'iGABA': {'pointp':'synGABA', 'var':'i'},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -70.038
h.tstop = 2000.0
h.init()
h.run()

In [ ]:
# Plot recorded traces
plot_interval = (700, 1000)

plt.figure()
v_soma = vec_dict['V_soma'].as_numpy()
t_soma = np.arange(len(v_soma)) * rec_dt
plt.plot(t_soma, v_soma)
#plt.ylim((-41, -55))
plt.xlim(plot_interval)
plt.grid(True)

In [ ]:
# Plot synaptic traces
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['gGABAA'].as_numpy(), 'b', label='gGABA-A')
ax.plot(t_soma, vec_dict['gGABAB'].as_numpy(), 'r', label='gGABA-B')
ax.set_ylabel('conductance (uS)')
ax.set_xlim(plot_interval)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['iGABAA'].as_numpy(), 'b', label='iGABA-A')
ax.plot(t_soma, vec_dict['iGABAB'].as_numpy(), 'r', label='iGABA-B')
ax.plot(t_soma, vec_dict['iGABA'].as_numpy(), 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.set_xlim(plot_interval)
ax.legend()

# Plot vesicle pool dynamcis
fig, axes = plt.subplots(2,1)

ax = axes[0]
ax.plot(t_soma, vec_dict['Rrp'].as_numpy(), 'b', label='Rrp')
ax.plot(t_soma, vec_dict['Use'].as_numpy(), 'r', label='Use')
ax.set_xlim(plot_interval)
ax.grid(True)
ax.legend()

ax = axes[1]
ax.plot(t_soma, vec_dict['gGABAA'].as_numpy(), 'g', label='gGABA-A')
ax.set_xlim(plot_interval)
ax.grid(True)
ax.legend()

# All Inputs

The Observations Database contains following observations that may help in configuring the distribution of synapses on the STN dendritic tree:

## Experimental Data

- Afferents and prsynaptic firing rate firing rate
- Postsynaptic firing rate
- Ratio of inputs

### Guzman (2003)

Article p. 7:

> In voltage-clamp mode, unitary IPSCs range from 20 to 250 pA (Koos et al., 2002); the minimum is in the range of quantal events in other synapses (Hanse and Gustafsson, 2001). With our stimulation parameters, the average synaptic current, 67 pA (10-250 pA), was in the unitary range. There- fore, an average of five boutons per connection is a reasonable number (quantum, 10-20 pA, depending on intracellular Cs+). Similar stimulation parameters yielded an average IPSC of 93 pA (20-500 pA) for intrastriatal inhibition, suggesting that axon collateral inhibition is quantitatively less than interneuron inhibition; 

- Average of five boutons per afferent onto MSN

> however, a spiny neuron receives  10,000 asymmetrical (Kincaid et al., 1998) and  2500 symmetrical synapses (Ingham et al., 1998). Symmetrical synapses can be dopaminergic, cholinergic, or GABAergic, with dopaminergic being  13% (Roberts et al., 2002). Assuming that cholinergic inputs are a similar percentage, 650 symmetrical synapses per spiny cell are not GABAergic. This leaves 1850 GABAergic synapses per spiny cell. How many come from axon collaterals? There are  2840 striatal neurons inside the volume of a spiny dendritic tree (Oorschot, 1996; Kincaid et al., 1998). Most spiny neurons have their axon collaterals restricted to this volume (Kawaguchi et al., 1990). If 95% of the neurons are spiny, the number of potentially contacting neurons onto a single centered spiny cell is  2700. Only  10% of neighboring spiny neurons are connected with one another (Czubayko and Plenz, 2002; Tunstall et al., 2002); therefore, connecting spiny neurons total only  270. An average of five contacts yields 1350 inhibitory synapses per spiny cell arising from axon collaterals: two-thirds of the GABAergic inputs.

- 10.000 asymmetrical synapses per MSN
    + (usually correspond to excitatory synapses)


- 2.500 symmetrical synapses per MSN
    + can be DAergic, ACHergic, or GABAergic
    + 13% of these are DAergic
    + 13% of these are cholinergic (extrapolation, no data)
    + => 74% = 1.850 synapses are GABAergic


- 2.700 candidate pre-synaptic MSN can contact dendritic tree of target MSN
    + 10% of neighboring MSN are connected to eachother
    + => 0.1 * 2.700 = 270 MSN afferents to MSN (afferent colaterals)
    + => 5 * 270 = 1.350 contacts (boutons) from those 270 MSN afferents
    + => 1.850 - 1.350 = 500 GABAergic contacts/synapses come from other inhibitory sources

> A similar calculation for interneurons explains the remaining contacts: 5% of the surrounding 2840 striatal neurons are interneurons, which is  140. One-fourth of the surrounding spiny cells receive inputs from a centered interneuron (Koos and Tepper, 1999), which leaves  30 interneurons converging in a single spiny cell. Fast- and low-threshold spiking interneurons might leave up to 15 (range, 8–15) contacts per spiny cell (Kubota and Kawaguchi, 2000). Thus, 30 x 15 = 500 contacts from interneurons. The later calculation has experimental support: 4-27 FS interneurons may converge on a single projection cell (Koos and Tepper, 1999). This makes an average of 16 interneurons of a single class converging on a spiny cell, which makes 16 x 15 = 240 contacts from one class of interneuron. Assuming the same number of contacts from both GABAergic interneuron genres (Kubota and Kawaguchi, 2000), the number of interneuronal contacts is again  500. Thus, 70% of all GABAergic contacts are from axon collaterals, and 30% are from interneurons.

- __NOTE: if connection probability (same in both directions per definition) is X%, then the number of inputs on a post-synaptic cell is X% * N_PRE and the number of targets of a pre-synaptic cell is X% * N_POST__
- 30 interneurons converge on single MSN (rough estimate)
- 4-27 interneurons converge on single MSN (experimental data)
- 8-15 contacts per interneuron (both FS and LTS)
- => 30 x 15 = 500 contacts per MSN coming from interneurons
- 16 neurons of a single class converging on an MSN (FS, LTS, Ach)
- => 16 x 15 = 240 contacts per MSN coming from one interneuron class

### Steiner, Tseng (2017)

- 2,800,000 = number of MSN in rat, unilaterally
- 17,000,000 = nuber of cortico-striatal projection neurons in rat, unilaterally
- number of interneurons in rat, unilaterally
    + 21,300 GABAergic SOM/NPY/NOS expressing interneurons
    + 16,900 GABAergic PV expressing interneurons
    + 13,200 GABAergic Calretinin expressing interneurons
    + 12,200 ACHergic interneurons
- 46,000 = number of GPe _projection_ neurons in rat, unilaterally

### Corbit (2017)

- 40 MSN cells
- 8 FSI cells
- 8 GPe cells

| Pre | Post | P_conn       | N_pre (N*P_conn) | gsyn (mS/cm2)    | tau_r | tau_d |
|-----|------|--------------|------------------|------------------|-------|-------|
| FSI | MSN  | 37.5 (DD:75) | 3 (DD: 6)        | 0.12 (code:0.15) | 0.5   | 4.35  |
| MSN | MSN  | 35           | 14               | 0.09 (code:0.14) | 0.5   | 10.0  |
| GPe | MSN  | 37.5         | 0 (Fig.4: 3)     | 0.003 (DD:0.01)  | 0.5   | 8.3   |
| CTX | MSN  | 100.0        | 1 (passive)      | 0.066 (DD:0.083) | 0     | 0     |

## Final Parameters

Each model synapse will represent all the contacts by a single afferent cell.

- MSN->MSN
    + 270 MSN afferents * 5 contacts/afferent = __1.350 contacts__ per MSN coming __from other MSN__
    + => __270 model synapses from MSN__


- IN->MSN:
    - 500 GABAergic contacts _not_ coming from MSNs
    - 4-27 (experimental) or 30 (calculated) interneurons converge on single MSN
    - 16 neurons of a single class converging on an MSN (FS, LTS, Ach)
    - 8-15 contacts per interneuron (both FS and LTS)
    - => 30 x 15 = __500 contacts__ per MSN coming __from interneurons__
    - => 16 x 15 = __240 contacts__ per MSN coming __from one interneuron class__ (FS/LTS/Ach)
    - => __16 model synapses from FS in CONTROL__ (GABAergic)
    - => __32 model synapses from FS in DD__
    - => __16 model synapses from LTS__ (GABAergic)
    - => __16 model synapses from ACH__ (ACHergic)


- CTX->MSN
    - 10.000 asymmetrical synapses per MSN (*Guzman (2003)*)
        + usually correspond to excitatory synapses
        + => __10.000 contacts coming from CTX__
    - each cortico-striatal axon makes only a single or a few contacts with a single MSN
        + source: *Steiner, Tseng (2017) - Ch. 1*
        + => __10.000 / 2 = 5.000 model synapses from CTX__
    - no two striatal MSN share common inputs from the cortex
        + this would imply we need NUM_MSN * 5000 independent spike trains

The ratios of inputs for `CTX:MSN:FS` with real population sizes and spike rates is:
- in __CONTROL__ - 5000:270:16
- in __DD__ - 5000:270:32

Now we can find suitable scaling laws, for an MSN population size of 100. The question is: what is important to conserve? Intuitively, it makes more sense to conserve converge/divergence patterns (connection probabilities) rather than the ratio of inputs on a single cell, or the ratio of population sizes. We can then scale the magnitude of conductances to preserve the original ratio of total conductance. The argument is that an unrealistic number of shared inputs will connect the network more tightly. This is different from Corbit (2016) which adjusts connection probabilities to conserve ratio of inputs.

- contacts from 10% of its neighboring MSN => __10 model synapses from MSN__
    + 270/10 = 27 => __`gsyn_msn` scale factor = 27.0__
- if we make 20 FSI and follow the observed 25% connection probability between neighboring MSN-IN:
    + => 20 * 0.25 = __5 model synapses from MSI__
    + => 16/5 = 3.2 => __`gsyn_fs` scale factor = 3.2__
- we are constrained by the fact that no two striatal MSN share common inputs from cortex
    + if we use N independent spike trains at K times the rate of CTX the reduction factor is `5000 / (K*N)` 
    + => __`gsyn_ctx` scale factor = 5000/(K*N)__

## ALL -> MSN (GABA + GLU)

In [ ]:
# Clear old inputs
stim_data = {}
istim = None

# Parameters for afferents
afferent_params = aff_par = {}

aff_par['MSN'] = {}
aff_par['MSN']['nsyn'] = NUM_PRE_MSN = 10
aff_par['MSN']['rate'] = RATE_MSN = (1.0 - 0.0) / 2
aff_par['MSN']['gsyn_scale'] = 270.0 / NUM_PRE_MSN

aff_par['FSI'] = {}
aff_par['FSI']['nsyn'] = NUM_PRE_FSI = 5
aff_par['FSI']['rate'] = RATE_FSI = (100.0 - 10.0) / 2
aff_par['FSI']['gsyn_scale'] = SCALE_GSYN_FSI = 16.0 / 5


aff_par['CTX'] = {}
aff_par['CTX']['nsyn'] = NUM_PRE_CTX = 10
aff_par['CTX']['rate_scale'] = SCALE_RATE_CTX = 5
aff_par['CTX']['rate'] = RATE_CTX = (5.0 - 1.0) / 2 * SCALE_RATE_CTX
NUM_PRE_CTX_CALIBRATION = 60 # estimate of number of afferent activated in experiment Ding (2010)
NUM_PRE_CTX_RAT_TOTAL = 5000.0 # rat, unilaterally, assuming 2 contacts per afferent
# - first we calculate ratio of actual number of inputs / number of inputs that our synapses represent
# - then we correct for fact that gsyn_ctx was calibrated not for response to simulatenously activated afferents
aff_par['CTX']['gsyn_scale'] = NUM_PRE_CTX_RAT_TOTAL / (NUM_PRE_CTX * SCALE_RATE_CTX) / NUM_PRE_CTX_CALIBRATION

print("CTX gsyn scale = {}".format(aff_par['CTX']['gsyn_scale']))


# First simulate without bursty inputs
afferent_constructors = {
    'MSN': make_msn_afferents,
    'FSI': make_fsi_afferents,
    'CTX': make_ctx_afferents,
}
# for pre in afferent_params.keys():
#     make_afferents = afferent_constructors[pre]
#     make_afferents(aff_par[pre]['nsyn'], rate=aff_par[pre]['rate'],
#                    scale_gsyn=aff_par[pre]['gsyn_scale'], noise=1)

make_msn_afferents(NUM_PRE_MSN, rate=RATE_MSN, scale_gsyn=aff_par['MSN']['gsyn_scale'], noise=1.0, tstart=20)
make_fsi_afferents(NUM_PRE_FSI, rate=RATE_FSI, scale_gsyn=aff_par['FSI']['gsyn_scale'], noise=1.0, tstart=20)
make_ctx_afferents(NUM_PRE_CTX, rate=RATE_CTX, scale_gsyn=aff_par['CTX']['gsyn_scale'], noise=1.0, tstart=20,
                   intraburst_rate=100, interburst_interval=500.0, burst_duration=50.0, burst_noise=1.0)

# Check that we have correct number of inputs
for pre in stim_data.keys():
    print("Afferents from {pre} have:\n\t- {nsyn} synapses"
          "\n\t- {nstim} spike generators".format(
              pre=pre, nsyn=len(stim_data[pre]['synapses']),
              nstim=len(stim_data[pre]['netstims'])))

## Record & Simulate

In [ ]:
# Define traces
rec_secs = {
    'soma': soma,
    'soma_spiker': h.NetCon(soma(0.5)._ref_v, None, -10.0, 0, 0),
#     'prox': proximal_seg,
#     'mid': middle_seg,
#     'dist': distal_seg,
#     'synGABA': synapses[0],
}

trace_specs = {
    't_global': {'var': 't'},
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    'AP_soma': {'netcon': 'soma_spiker'},
    # Synapse variables (pointp added below)
    'gAMPA': {'pointp':'synCTX0', 'var':'g_AMPA'},
    'gNMDA': {'pointp':'synCTX0', 'var':'g_NMDA'},
    'iAMPA': {'pointp':'synCTX0', 'var':'i_AMPA'},
    'iNMDA': {'pointp':'synCTX0', 'var':'i_NMDA'},
    'iGLU':  {'pointp':'synCTX0', 'var':'i'},
#     'Rrp': {'pointp':'synGLU', 'var':'R'},
#     'Use': {'pointp':'synGLU', 'var':'Use'},
    'gGABAA': {'pointp':'synFSI0', 'var':'g_GABAA'},
    'gGABAB': {'pointp':'synFSI0', 'var':'g_GABAB'},
    'iGABAA': {'pointp':'synFSI0', 'var':'i_GABAA'},
    'iGABAB': {'pointp':'synFSI0', 'var':'i_GABAB'},
    'iGABA':  {'pointp':'synFSI0', 'var':'i'},
    'Gprot':  {'pointp':'synFSI0', 'var':'G'},
}

# Record ALL spikes and synaptic currents
for pop in 'CTX', 'MSN', 'FSI':
    for i_syn, nc in enumerate(stim_data[pop]['netcons']):
        con_label = "con{}{}".format(pop, i_syn)
        syn_label = "syn{}{}".format(pop, i_syn)
        rec_secs[con_label] = nc
        rec_secs[syn_label] = nc.syn()
        trace_specs['AP_'+con_label] = {'netcon': con_label}
        trace_specs['i_'+syn_label] = {'pointp': syn_label, 'var':'i'}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -70.038
h.tstop = 5000.0
h.init()

import time
tstart = time.time()
h.run()

tstop = time.time()
cputime = tstop - tstart
num_segments = sum((sec.nseg for sec in h.allsec()))
print("Simulated {} segments for {} ms in {} ms CPU time".format(
        num_segments, h.tstop, cputime))

## Plot & Analyze

### Plot Voltage Traces

In [ ]:
# Plot recorded traces
# t_global = vec_dict.pop('t_global').as_numpy() # inconsistent if dt != rec_dt 
interval = (2000, h.tstop)

# Plot somatic voltage traces
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt
# plt.figure()
# plt.plot(t_rec, v_soma)
# plt.grid(True)
spikes_soma = vec_dict['AP_soma'].as_numpy()
spike_range = (spikes_soma > interval[0]) & (spikes_soma <= interval[1])
soma_rate = len(spikes_soma[spike_range]) / float(interval[1]-interval[0]) * 1e3
analysis.plotTraces(vec_dict, rec_dt, interval, includeTraces=['V_soma'], title="Soma @ {} Hz".format(soma_rate))

### Plot Spike Trains

In [ ]:
# Plot spikes
spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conCTX'))
analysis.plotRaster(spike_data, interval, color='b', title='{} CTX spiketrains'.format(len(spike_data)))

spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conMSN'))
analysis.plotRaster(spike_data, interval, color='r', title='{} MSN spiketrains'.format(len(spike_data)))

spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP_conFSI'))
analysis.plotRaster(spike_data, interval, color='r', title='{} FSI spiketrains'.format(len(spike_data)))

### Plot Synaptic Currents

In [ ]:
# Plot synaptic variables
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

# Plot GLU synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gAMPA'].as_numpy()[a:b], 'b', label='gAMPA')
ax.plot(t_plot, vec_dict['gNMDA'].as_numpy()[a:b], 'r', label='gNMDA')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GLU Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iAMPA'].as_numpy()[a:b], 'b', label='iAMPA')
ax.plot(t_plot, vec_dict['iNMDA'].as_numpy()[a:b], 'r', label='iNMDA')
ax.plot(t_plot, vec_dict['iGLU'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.legend()
ax.grid(True)

In [ ]:
# Plot GABA synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gGABAA'].as_numpy()[a:b], 'b', label='gGABAA')
ax.plot(t_plot, vec_dict['gGABAB'].as_numpy()[a:b], 'r', label='gGABAB')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GABA Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iGABAA'].as_numpy()[a:b], 'b', label='iGABAA')
ax.plot(t_plot, vec_dict['iGABAB'].as_numpy()[a:b], 'r', label='iGABAB')
ax.plot(t_plot, vec_dict['iGABA'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.grid(True)
ax.legend()

In [ ]:
# G protein concentration
# fig, ax = plt.subplots(1, 1, figsize=(10,4))
# ax.plot(t_plot, vec_dict['Gprot'].as_numpy()[a:b], 'b', label='Gprot')
# ax.set_ylabel('G protein')
# ax.legend()
# ax.grid(True)
# ax.set_title('G protein concentration')

# Plot total GABA synapse current
itot_GABA = vec_dict['i_synFSI0'].as_numpy()
for i in range(1, len(stim_data['FSI']['synapses'])):
    itot_GABA += vec_dict['i_synFSI{}'.format(i)].as_numpy()
for i in range(0, len(stim_data['MSN']['synapses'])):
    itot_GABA += vec_dict['i_synMSN{}'.format(i)].as_numpy()

itot_GLU = vec_dict['i_synCTX0'].as_numpy() * -1
for i in range(1, len(stim_data['CTX']['synapses'])):
    itot_GLU -= vec_dict['i_synCTX{}'.format(i)].as_numpy()

plt.figure(figsize=(10,4))
plt.plot(t_rec, itot_GABA, 'r', label='i_GABA')
plt.plot(t_rec, itot_GLU, 'b', label='i_GLU')
plt.ylabel('current (nA)')
plt.xlabel('time (ms)')
plt.legend()
plt.grid()
plt.show()
plt.suptitle('Total GABA and GLU current')
# Plot remaining traces
# figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),